LEAARNING ABOUT THE PARQUT FILE FORMAT 


In [54]:
import pyspark
# from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
spark=SparkSession.builder.appName("parquet-exp.com").getOrCreate()

24/09/11 10:28:50 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [27]:
data=[
    (("James","","Smith"),["Java","Scala","C++"],"OH","M"),
    (("Anna","Rose",""),["Spark","Java","C++"],"NY","F"),
    (("Julia","","Williams"),["CSharp","VB"],"OH","F"),
    (("Maria","Anne","Jones"),["CSharp","VB"],"NY","M"),
    (("Jen","Mary","Brown"),["CSharp","VB"],"NY","M"),
    (("Mike","Mary","Williams"),["Python","VB"],"OH","M")
 ]

In [28]:
columns=["name","languages","state","Gender"]
df=spark.createDataFrame(data=data,schema=columns)
df.show(5)

+--------------------+------------------+-----+------+
|                name|         languages|state|Gender|
+--------------------+------------------+-----+------+
|    {James, , Smith}|[Java, Scala, C++]|   OH|     M|
|      {Anna, Rose, }|[Spark, Java, C++]|   NY|     F|
| {Julia, , Williams}|      [CSharp, VB]|   OH|     F|
|{Maria, Anne, Jones}|      [CSharp, VB]|   NY|     M|
|  {Jen, Mary, Brown}|      [CSharp, VB]|   NY|     M|
+--------------------+------------------+-----+------+
only showing top 5 rows



In [29]:
df.write.parquet("/home/ai/Downloads/Pyspark_learning/sampleparquet.parquet")

In [30]:
df2=spark.read.parquet("/home/ai/Downloads/Pyspark_learning/sampleparquet.parquet")
df2.show()

+--------------------+------------------+-----+------+
|                name|         languages|state|Gender|
+--------------------+------------------+-----+------+
|{Mike, Mary, Will...|      [Python, VB]|   OH|     M|
|{Maria, Anne, Jones}|      [CSharp, VB]|   NY|     M|
| {Julia, , Williams}|      [CSharp, VB]|   OH|     F|
|  {Jen, Mary, Brown}|      [CSharp, VB]|   NY|     M|
|    {James, , Smith}|[Java, Scala, C++]|   OH|     M|
|      {Anna, Rose, }|[Spark, Java, C++]|   NY|     F|
+--------------------+------------------+-----+------+



In [31]:
dataDF = [(('James','','Smith'),'1991-04-01','M',3000),
  (('Michael','Rose',''),'2000-05-19','M',4000),
  (('Robert','','Williams'),'1978-09-05','M',4000),
  (('Maria','Anne','Jones'),'1967-12-01','F',4000),
  (('Jen','Mary','Brown'),'1980-02-17','F',-1)
  
]

In [32]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
schema = StructType([
        StructField('name', StructType([
             StructField('firstname', StringType(), True),
             StructField('middlename', StringType(), True),
             StructField('lastname', StringType(), True)
             ])),
         StructField('dob', StringType(), True),
         StructField('gender', StringType(), True),
         StructField('Salary', IntegerType(), True)
         ])

In [34]:

# from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('parquetExamples.com').getOrCreate()
df3 = spark.createDataFrame(data = dataDF, schema = schema)
df3.printSchema()
df3.show()

root
 |-- name: struct (nullable = true)
 |    |-- firstname: string (nullable = true)
 |    |-- middlename: string (nullable = true)
 |    |-- lastname: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- Salary: integer (nullable = true)

+--------------------+----------+------+------+
|                name|       dob|gender|Salary|
+--------------------+----------+------+------+
|    {James, , Smith}|1991-04-01|     M|  3000|
|   {Michael, Rose, }|2000-05-19|     M|  4000|
|{Robert, , Williams}|1978-09-05|     M|  4000|
|{Maria, Anne, Jones}|1967-12-01|     F|  4000|
|  {Jen, Mary, Brown}|1980-02-17|     F|    -1|
+--------------------+----------+------+------+



appending the data from the new dataframe to the existing file

In [35]:
df3.write.parquet("/home/ai/Downloads/Pyspark_learning/sampleparquet1.parquet")

In [40]:
df2.write.mode("append").parquet("/home/ai/Downloads/Pyspark_learning/sampleparquet1.parquet")
df2.show()

+--------------------+------------------+-----+------+
|                name|         languages|state|Gender|
+--------------------+------------------+-----+------+
|{Mike, Mary, Will...|      [Python, VB]|   OH|     M|
|{Maria, Anne, Jones}|      [CSharp, VB]|   NY|     M|
| {Julia, , Williams}|      [CSharp, VB]|   OH|     F|
|  {Jen, Mary, Brown}|      [CSharp, VB]|   NY|     M|
|    {James, , Smith}|[Java, Scala, C++]|   OH|     M|
|      {Anna, Rose, }|[Spark, Java, C++]|   NY|     F|
+--------------------+------------------+-----+------+



the file has been appended since it has same schema but the stuctfield is different so it turns out to be null

Spark sql query

In [45]:
df3.createOrReplaceTempView("sampleParquet1")

In [47]:
parksql=spark.sql("select * from sampleParquet1 where salary>=4000")

In [49]:
parksql.show()

+--------------------+----------+------+------+
|                name|       dob|gender|Salary|
+--------------------+----------+------+------+
|   {Michael, Rose, }|2000-05-19|     M|  4000|
|{Robert, , Williams}|1978-09-05|     M|  4000|
|{Maria, Anne, Jones}|1967-12-01|     F|  4000|
+--------------------+----------+------+------+



In [52]:
spark.sql("CREATE TEMPORARY VIEW PERSONs USING parquet OPTIONS (path \"/home/ai/Downloads/Pyspark_learning/sampleparquet1.parquet\")")


DataFrame[]

In [59]:
# df3.select(explode(df3.name),df3.dob,df3.gender,df3.Salary)
df3.dtypes


[('name', 'struct<firstname:string,middlename:string,lastname:string>'),
 ('dob', 'string'),
 ('gender', 'string'),
 ('Salary', 'int')]

In [61]:
data =[("James ","","Smith","36636","M",3000),
              ("Michael ","Rose","","40288","M",4000),
              ("Robert ","","Williams","42114","M",4000),
              ("Maria ","Anne","Jones","39192","F",4000),
              ("Jen","Mary","Brown","","F",-1)]
columns=["firstname","middlename","lastname","dob","gender","salary"]
df4=spark.createDataFrame(data,columns)
# <!-- above example, it creates a DataFrame with  -->

In [62]:
df4.write.parquet("/home/ai/Downloads/Pyspark_learning/partion.parquet")

In [63]:
df4.write.partitionBy("gender","salary").mode("overwrite").parquet("/home/ai/Downloads/Pyspark_learning/partion.parquet")

In [64]:
parDF2=spark.read.parquet("/home/ai/Downloads/Pyspark_learning/partion.parquet")
parDF2.show(truncate=False)

+---------+----------+--------+-----+------+------+
|firstname|middlename|lastname|dob  |gender|salary|
+---------+----------+--------+-----+------+------+
|Maria    |Anne      |Jones   |39192|F     |4000  |
|Robert   |          |Williams|42114|M     |4000  |
|Michael  |Rose      |        |40288|M     |4000  |
|James    |          |Smith   |36636|M     |3000  |
|Jen      |Mary      |Brown   |     |F     |-1    |
+---------+----------+--------+-----+------+------+



In [68]:
spark.sql("CREATE TEMPORARY VIEW PERSON2 USING parquet OPTIONS(path \"/home/ai/Downloads/Pyspark_learning/partion.parquet\")")

DataFrame[]

In [71]:
spark.sql("select * from PERSON2").show()

+---------+----------+--------+-----+------+------+
|firstname|middlename|lastname|  dob|gender|salary|
+---------+----------+--------+-----+------+------+
|   Maria |      Anne|   Jones|39192|     F|  4000|
|  Robert |          |Williams|42114|     M|  4000|
| Michael |      Rose|        |40288|     M|  4000|
|   James |          |   Smith|36636|     M|  3000|
|      Jen|      Mary|   Brown|     |     F|    -1|
+---------+----------+--------+-----+------+------+

